# 一、机器翻译  

机器翻译是指利用计算机技术来将一种自然语言的文本翻译成另一种自然语言的文本。这通常涉及使用大量的语言学知识和统计学习算法来创建模型，让计算机能够理解文本的含义并将其准确地转换成另一种语言。机器翻译系统通常包括以下步骤：

文本预处理：包括分词、词性标注和句法分析等，以便计算机能够理解和处理输入的文本。

特征提取：通过将文本转换成计算机可理解的向量形式，以表示每个词或短语在不同语言中的含义和上下文。

模型训练：利用大量的双语对照文本来训练机器学习模型，包括统计机器翻译模型、神经机器翻译模型等。

评估与优化：使用不同的评估指标来评估翻译系统的性能，例如BLEU、METEOR等，并根据评估结果对模型进行优化和改进。  

以机器翻译为例，介绍编码器—解码器和注意力机制的应用。

## 1.1 读取和预处理数据 ##
首先定义一些特殊符号：  
\<pad\>: 用于将序列填充至相同长度，因为在训练过程中经常会遇到长度不一的序列，为了构建一个批次进行训练，我们需要将短序列填充至与最长序列相同的长度。

\<bos\>: 表示序列的开始。

\<eos\>: 表示序列的结束。

In [1]:
!tar -xf d2lzh_pytorch.tar
import collections
import os
import io
import math
import torch
from torch import nn
import torch.nn.functional as F
import torchtext.vocab as Vocab
import torch.utils.data as Data
import sys
import d2lzh_pytorch as d2l


PAD, BOS, EOS = '<pad>', '<bos>', '<eos>'
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(torch.__version__, device)

2.3.0+cpu cpu


定义两个辅助函数,对读取的数据进行预处理。  
process_one_seq 函数的作用是处理单个序列。它将该序列中的词加入到所有词的列表中，然后在该序列后面添加结束符EOS，并用PAD填充该序列至指定的最大长度max_seq_len，最后将处理后的序列加入到所有序列的列表中。

build_data 函数的作用是利用所有的词来构造词典，并将所有序列中的词转换为词索引后构造成Tensor。首先使用所有的词构建词典，包括特殊符号PAD, BOS, EOS。然后将所有序列中的词转换为在词典中的索引，并构建成Tensor，最终返回构建好的词典和对应的Tensor。

In [2]:
# 将一个序列中所有的词记录在all_tokens中以便之后构造词典
# 然后在该序列后面添加PAD直到序列长度变为max_seq_len
# 然后将序列保存在all_seqs中
def process_one_seq(seq_tokens, all_tokens, all_seqs, max_seq_len):
    # 将该序列中的词加入到所有词的列表中
    all_tokens.extend(seq_tokens)
    # 在该序列后面添加EOS结束符，然后用PAD填充至max_seq_len长度
    seq_tokens += [EOS] + [PAD] * (max_seq_len - len(seq_tokens) - 1)
    # 将处理后的序列加入到所有序列的列表中
    all_seqs.append(seq_tokens)

# 使用所有的词来构造词典。并将所有序列中的词变换为词索引后构造Tensor
def build_data(all_tokens, all_seqs):
    # 使用所有的词构建词典，包括特殊符号PAD, BOS, EOS
    vocab = Vocab.Vocab(collections.Counter(all_tokens), specials=[PAD, BOS, EOS])
    # 将所有序列中的词转换为在词典中的索引，并构建为Tensor
    indices = [[vocab.stoi[w] for w in seq] for seq in all_seqs]
    return vocab, torch.tensor(indices)



我们有一个包含法语句子和对应的英语句子的小型数据集，接下来将利用它来进行演示。   

在这个数据集里，每一行是一对法语句子和它对应的英语句子，中间使用制表符'\t'隔开。   

我们会在每个句子的末尾添加一个“\<eos\>”表示句子结束，并且可能会使用“\<pad\>”来填充句子，使得每个序列的长度都变为max_seq_len。  
  
我们会为法语词和英语词分别创建词典，这意味着我们会为每种语言单独构建一个词汇表，并且这两个词汇表是相互独立的，即法语词的索引和英语词的索引是相互独立的。   


![](https://p.sda1.dev/18/e65832173d3c6bd5e86e2a35238e043c/英法数据集.png)
<font color='grey'><center>图1.1 法语—英语数据集</font></center>

In [3]:
def read_data(max_seq_len):
    # in和out分别是input和output的缩写
    in_tokens, out_tokens, in_seqs, out_seqs = [], [], [], []  # 初始化四个空列表，用来存储输入和输出的tokens和sequences
    with io.open('fr-en-small.txt') as f:  # 打开文件
        lines = f.readlines()  # 逐行读取文件内容
    for line in lines:  # 遍历每一行
        in_seq, out_seq = line.rstrip().split('\t')  # 将每一行按制表符分割为输入序列和输出序列
        in_seq_tokens, out_seq_tokens = in_seq.split(' '), out_seq.split(' ')  # 将输入序列和输出序列按空格分割为tokens
        if max(len(in_seq_tokens), len(out_seq_tokens)) > max_seq_len - 1:  # 如果加上EOS后长于max_seq_len，则忽略掉此样本
            continue  
        process_one_seq(in_seq_tokens, in_tokens, in_seqs, max_seq_len)  # 处理输入序列的tokens，并添加到相应的列表中
        process_one_seq(out_seq_tokens, out_tokens, out_seqs, max_seq_len)  # 处理输出序列的tokens，并添加到相应的列表中
    in_vocab, in_data = build_data(in_tokens, in_seqs)  # 使用处理好的输入tokens和sequences构建输入词典和数据Tensor
    out_vocab, out_data = build_data(out_tokens, out_seqs)  # 使用处理好的输出tokens和sequences构建输出词典和数据Tensor
    return in_vocab, out_vocab, Data.TensorDataset(in_data, out_data)  # 返回构建好的输入词典、输出词典和对应的数据集Tensor


在read_data()函数中：  
对每一行数据，分割输入序列和输出序列，并将它们分别处理为tokens。  
如果输入序列和输出序列中的 tokens 加上 EOS 后的长度超过指定的最大长度 max_seq_len，则忽略这个样本。  
调用定义的 process_one_seq 函数来处理输入序列和输出序列的 tokens，并将它们添加到相应的列表中。  
调用定义的 build_data 函数使用处理好的 tokens 和 sequences 构建词典和数据 Tensor。  
最后返回构建好的输入词典、输出词典和对应的数据集 Tensor。 

将序列的最大长度设成7，然后查看读取到的第一个样本。该样本分别包含法语词索引序列和英语词索引序列。

In [4]:
max_seq_len = 7
in_vocab, out_vocab, dataset = read_data(max_seq_len)
dataset[0]

(tensor([ 5,  4, 45,  3,  2,  0,  0]), tensor([ 8,  4, 27,  3,  2,  0,  0]))

## 1.2 含注意力机制的编码器—解码器 ## 
使用含注意力机制的编码器—解码器来将一段简短的法语翻译成英语。

### 1.2.1 编码器 ###
在编码器中，我们首先将输入语言的词索引通过词嵌入层转换为词的表征（也就是词嵌入向量），这样就可以用向量的形式表示每个单词。接着，将这些词嵌入向量输入到一个多层门控循环单元（GRU）中，以便对输入序列进行建模和提取特征。

在使用PyTorch的nn.GRU实例时，它在完成前向计算后会返回两个主要结果：输出和最终时间步的多层隐藏状态。这里的输出指的是最后一层GRU的隐藏层在每个时间步的隐藏状态，这些隐藏状态用于捕捉输入序列的信息。需要注意的是，这些输出并不直接涉及到输出层的计算，而主要是用于编码器中的特征提取。

在注意力机制中，这些编码器的输出被用作注意力机制的"键项"和"值项"。在注意力机制中，这些隐藏状态会被用来计算注意力权重，以便在解码阶段中对不同时间步的编码器隐藏状态进行加权组合，从而更好地关注输入句子中与当前要翻译的单词相关的部分。

In [5]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers,
                 drop_prob=0, **kwargs):
        super(Encoder, self).__init__(**kwargs)
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.GRU(embed_size, num_hiddens, num_layers, dropout=drop_prob)

    def forward(self, inputs, state):
        # 输入形状是(批量大小, 时间步数)。将输出互换样本维和时间步维
        embedding = self.embedding(inputs.long()).permute(1, 0, 2) # (seq_len, batch, input_size)
        # 将嵌入向量经过词嵌入层并进行维度变换，使其适应GRU层的输入要求
        return self.rnn(embedding, state)

    def begin_state(self):
        # 返回初始状态，这里为None，因为在Encoder中并不需要初始化状态
        return None


下面的例子创建了一个批量大小为4、时间步数为7的小批量序列输入。  
接着定义了门控循环单元（GRU）的隐藏层个数为2，隐藏单元个数为16。当编码器对该输入执行前向计算后，返回的输出形状将是(7, 4, 16)，即(时间步数, 批量大小, 隐藏单元个数)。  
对于门控循环单元在最终时间步的多层隐藏状态的形状为(2, 4, 16)，即(隐藏层个数, 批量大小, 隐藏单元个数)。  
对于门控循环单元和长短期记忆单元，state的形状和内容会有所不同，因为GRU的state是一个隐藏状态元素，而LSTM的state是一个包含隐藏状态和记忆细胞的元组。

In [6]:
encoder = Encoder(vocab_size=10, embed_size=8, num_hiddens=16, num_layers=2)
output, state = encoder(torch.zeros((4, 7)), encoder.begin_state())
output.shape, state.shape # GRU的state是h, 而LSTM的是一个元组(h, c)

(torch.Size([7, 4, 16]), torch.Size([2, 4, 16]))

### 1.2.2 注意力机制 ###

注意力机制是一种用于在序列到序列模型中动态计算输入序列中不同部分之间相关性的一种技术。在这里，输入包括查询项、键项和值项。查询项代表解码器在上一时间步的隐藏状态，用来表示当前解码器所关注的内容。键项和值项则分别代表编码器在所有时间步的隐藏状态，用来表示编码器的输入序列。

具体来说，注意力机制使用查询项和键项的关联度来为值项中的每个元素分配一个权重，这个权重表示了值项中的信息对当前解码器状态的重要性。权重越大，代表该元素在当前时刻的解码过程中对结果的贡献越大。最后，通过对值项乘以相应的权重并相加，就得到了当前时间步的背景变量，它将会被用来辅助解码器生成当前时刻的输出。

因此，在这个情况下，注意力机制会基于解码器上一时间步的隐藏状态（查询项），以及编码器在所有时间步的隐藏状态（键项和值项），动态地计算当前时间步的背景变量，使其能够更好地捕捉输入序列中不同部分之间的相关性，并为解码器提供更准确的信息以生成当前时刻的输出。

实现attention_model函数。将输入数据连接后，通过一个隐藏层的多层感知机进行变换。 
隐藏层的输入是解码器的隐藏状态和编码器在所有时间步上的隐藏状态连接而成，然后使用tanh函数作为激活函数。  
输出层的输出个数为1。两个Linear实例（全连接层）都不使用偏差。在函数定义中，向量v 的长度是一个超参数，即attention_size。

In [7]:
import torch.nn as nn

def attention_model(input_size, attention_size):
    """
    创建一个简单的注意力模型，使用多层感知机实现。

    参数:
    input_size (int): 输入的大小。
    attention_size (int): 注意力向量的大小。

    返回:
    torch.nn.Sequential: 表示注意力机制的顺序模型。
    """
    model = nn.Sequential(
        nn.Linear(input_size, attention_size, bias=False),  # 线性变换到 attention_size
        nn.Tanh(),  # 双曲正切激活函数
        nn.Linear(attention_size, 1, bias=False)  # 线性变换到单个输出
    )
    return model

def attention_forward(model, enc_states, dec_state):
    """
    enc_states: (时间步数, 批量大小, 隐藏单元个数)
    dec_state: (批量大小, 隐藏单元个数)
    """
    # 将解码器隐藏状态广播到和编码器隐藏状态形状相同后进行连结
    dec_states = dec_state.unsqueeze(dim=0).expand_as(enc_states)
    enc_and_dec_states = torch.cat((enc_states, dec_states), dim=2)
    e = model(enc_and_dec_states)  # 形状为(时间步数, 批量大小, 1)
    alpha = F.softmax(e, dim=0)  # 在时间步维度做softmax运算
    return (alpha * enc_states).sum(dim=0)  # 返回背景变量

在下面的例子中，编码器的时间步数为10，批量大小为4，编码器和解码器的隐藏单元个数均为8。这表示我们有4个序列进行处理，并且每个序列中有10个时间步。每个时间步中，编码器和解码器的隐藏单元个数都是8，表示模型中的每个隐藏状态的维度为8。

由于注意力机制返回一个小批量的背景向量，每个背景向量的长度等于编码器的隐藏单元个数，因此输出的形状为(4, 8)。这意味着对于每个批量中的样本，注意力机制都会计算一个长度为8的背景向量，其中包含了编码器隐藏状态的信息，并且这些背景向量会被用于解码器的当前时间步来生成输出。

In [8]:
seq_len, batch_size, num_hiddens = 10, 4, 8
model = attention_model(2*num_hiddens, 10) 
enc_states = torch.zeros((seq_len, batch_size, num_hiddens))
dec_state = torch.zeros((batch_size, num_hiddens))
attention_forward(model, enc_states, dec_state).shape

torch.Size([4, 8])

### 1.2.3 含注意力机制的解码器 ###
首先，我们将编码器在最终时间步的隐藏状态作为解码器的初始隐藏状态。这意味着解码器将以编码器最终隐藏状态中的信息作为起点，来生成输出序列。

在解码器的前向计算中，首先利用注意力机制计算当前时间步的背景向量。这一背景向量将会捕捉编码器各个时间步隐藏状态的重要信息，有助于解码器在此刻进行正确的翻译。

解码器的输入是来自输出语言的词索引，经过词嵌入层得到表征。然后，将词嵌入的输出与背景向量在特征维度上进行连结，以确保当前时间步的输入能够充分利用背景向量的信息。

接下来，将连结后的结果与上一时间步的隐藏状态通过门控循环单元（比如GRU或者LSTM）进行计算，以得到当前时间步的输出与隐藏状态。这一步反映了解码器在当前时间步如何结合输入词嵌入和之前的隐藏状态来更新自身的状态。

最后，通过全连接层变换，将当前时间步的隐藏状态映射为有关各个输出词的预测概率。输出的形状为(批量大小, 输出词典大小)。这使得模型能够在每个时间步对输出词进行预测，从而最终生成整个目标语言序列。

In [9]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers,
                 attention_size, drop_prob=0):
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.attention = attention_model(2*num_hiddens, attention_size)
        # GRU的输入包含attention输出的c和实际输入, 所以尺寸是 num_hiddens+embed_size
        self.rnn = nn.GRU(num_hiddens + embed_size, num_hiddens, 
                          num_layers, dropout=drop_prob)
        self.out = nn.Linear(num_hiddens, vocab_size)

    def forward(self, cur_input, state, enc_states):
        """
        cur_input shape: (batch, )
        state shape: (num_layers, batch, num_hiddens)
        """
        # 使用注意力机制计算背景向量
        c = attention_forward(self.attention, enc_states, state[-1])
        # 将嵌入后的输入和背景向量在特征维连结, (批量大小, num_hiddens+embed_size)
        input_and_c = torch.cat((self.embedding(cur_input), c), dim=1) 
        # 为输入和背景向量的连结增加时间步维，时间步个数为1
        output, state = self.rnn(input_and_c.unsqueeze(0), state)
        # 移除时间步维，输出形状为(批量大小, 输出词典大小)
        output = self.out(output).squeeze(dim=0)
        return output, state

    def begin_state(self, enc_state):
        # 直接将编码器最终时间步的隐藏状态作为解码器的初始隐藏状态
        return enc_state

## 1.3 训练模型 ## 
我们先实现batch_loss函数计算一个小批量的损失。  
解码器在最初时间步的输入是特殊字符BOS（Beginning of Sequence），它表示解码器的开始。这是为了指示解码器开始生成目标序列。

在之后的时间步中，解码器的输入将是样本的输出序列中上一时间步的词，这种方式被称为"强制教学"（teacher forcing）。这意味着在训练过程中，解码器在每个时间步都会接收到样本输出序列中上一时间步的真实词作为输入，而不是使用自己生成的词作为输入。这有助于加快训练收敛速度，但在推理阶段需要额外的考虑。

与word2vec的实现类似，为了避免填充项对损失函数计算的影响，我们在这里也会使用掩码变量。这些掩码变量将会屏蔽掉填充部分的影响，确保损失函数的计算只考虑样本的实际部分。这有助于提高模型在处理变长序列时的效率，并且有效避免填充项对训练造成的干扰。

In [10]:
def batch_loss(encoder, decoder, X, Y, loss):
    batch_size = X.shape[0]
    enc_state = encoder.begin_state()
    enc_outputs, enc_state = encoder(X, enc_state)
    # 初始化解码器的隐藏状态
    dec_state = decoder.begin_state(enc_state)
    # 解码器在最初时间步的输入是BOS
    dec_input = torch.tensor([out_vocab.stoi[BOS]] * batch_size)
    # 我们将使用掩码变量mask来忽略掉标签为填充项PAD的损失, 初始全1
    mask, num_not_pad_tokens = torch.ones(batch_size,), 0
    l = torch.tensor([0.0])
    for y in Y.permute(1,0): # Y shape: (batch, seq_len)
        dec_output, dec_state = decoder(dec_input, dec_state, enc_outputs)
        l = l + (mask * loss(dec_output, y)).sum()
        dec_input = y  # 使用强制教学
        num_not_pad_tokens += mask.sum().item()
        # EOS后面全是PAD. 下面一行保证一旦遇到EOS接下来的循环中mask就一直是0
        mask = mask * (y != out_vocab.stoi[EOS]).float()
    return l / num_not_pad_tokens

在训练函数中，我们需要同时迭代编码器和解码器的模型参数。这是因为机器翻译模型是端到端的模型，编码器和解码器之间的参数是密切相关的。因此，在每个训练步骤中，我们都需要对编码器和解码器的参数进行更新，以便模型能够更好地适应训练数据，从而提高翻译质量。

In [11]:
def train(encoder, decoder, dataset, lr, batch_size, num_epochs):
    # 使用Adam优化器来分别优化编码器和解码器的参数
    enc_optimizer = torch.optim.Adam(encoder.parameters(), lr=lr)
    dec_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)

    # 定义交叉熵损失函数，reduction='none'表示不对损失进行平均，保留每个样本的损失值
    loss = nn.CrossEntropyLoss(reduction='none')

    # 创建一个数据迭代器，用于批量加载数据
    data_iter = Data.DataLoader(dataset, batch_size, shuffle=True)

    # 开始迭代训练过程
    for epoch in range(num_epochs):
        l_sum = 0.0  # 初始化损失总和
        # 遍历数据迭代器，获取批量的输入X和标签Y
        for X, Y in data_iter:
            # 每个批量数据前先将编码器和解码器的梯度清零
            enc_optimizer.zero_grad()
            dec_optimizer.zero_grad()
            # 计算当前批量数据的损失
            l = batch_loss(encoder, decoder, X, Y, loss)
            # 反向传播计算梯度
            l.backward()
            # 更新编码器和解码器的参数
            enc_optimizer.step()
            dec_optimizer.step()
            # 累加当前批量数据的损失值
            l_sum += l.item()
        # 每经过10个epoch，输出一次损失值
        if (epoch + 1) % 10 == 0:
            print("epoch %d, loss %.3f" % (epoch + 1, l_sum / len(data_iter)))


接下来，创建模型实例并设置超参数。然后，我们就可以训练模型了。  
参数包括嵌入大小（embed_size）、隐藏单元数（num_hiddens）、层数（num_layers）、注意力大小（attention_size）、dropout概率（drop_prob）、学习率（lr）、批量大小（batch_size）和训练迭代次数（num_epochs）。

In [12]:
# 定义编码器和解码器的相关参数
embed_size, num_hiddens, num_layers = 64, 64, 2
attention_size, drop_prob, lr, batch_size, num_epochs = 10, 0.5, 0.01, 2, 50

# 初始化编码器和解码器
encoder = Encoder(len(in_vocab), embed_size, num_hiddens, num_layers, drop_prob)
decoder = Decoder(len(out_vocab), embed_size, num_hiddens, num_layers, attention_size, drop_prob)

# 调用train函数开始训练
train(encoder, decoder, dataset, lr, batch_size, num_epochs)


epoch 10, loss 0.445
epoch 20, loss 0.176
epoch 30, loss 0.074
epoch 40, loss 0.029
epoch 50, loss 0.016


## 1.4 预测不定长的序列 ##   
在这里，我们实现最简单的贪婪搜索（greedy search）来进行不定长序列的预测。
贪婪搜索的基本思想是在每个时间步选择当前条件概率最大的词作为输出。换句话说，它会不断地选择最可能的下一个词，而不考虑整体句子的概率。

In [13]:
def translate(encoder, decoder, input_seq, max_seq_len):
    # 分割输入序列成token
    in_tokens = input_seq.split(' ')
    # 添加EOS（句子结束标记）和PAD（填充标记）并根据最大序列长度进行填充
    in_tokens += [EOS] + [PAD] * (max_seq_len - len(in_tokens) - 1)
    # 将token映射为词汇表索引，并构建编码器输入张量
    enc_input = torch.tensor([[in_vocab.stoi[tk] for tk in in_tokens]])  # batch=1
    # 初始化编码器状态
    enc_state = encoder.begin_state()
    # 编码器处理编码器输入，并得到编码器输出和最终状态
    enc_output, enc_state = encoder(enc_input, enc_state)
    # 初始化解码器输入为BOS（句子起始标记）
    dec_input = torch.tensor([out_vocab.stoi[BOS]])
    # 使用编码器状态初始化解码器状态
    dec_state = decoder.begin_state(enc_state)
    # 初始化输出tokens列表
    output_tokens = []
    # 循环生成输出序列
    for _ in range(max_seq_len):
        # 解码器处理当前输入和状态，并得到输出及更新状态
        dec_output, dec_state = decoder(dec_input, dec_state, enc_output)
        # 选择概率最高的预测token
        pred = dec_output.argmax(dim=1)
        pred_token = out_vocab.itos[int(pred.item())]
        # 如果预测的token为EOS，则停止生成序列
        if pred_token == EOS:
            break
        else:
            # 将预测token加入输出序列，并更新解码器输入为预测token
            output_tokens.append(pred_token)
            dec_input = pred
    # 返回生成的输出tokens
    return output_tokens


输入法语句子“ils regardent.”，翻译后的英语句子应该是“they are watching.”。

In [14]:
input_seq = 'ils regardent .'
translate(encoder, decoder, input_seq, max_seq_len)

['they', 'are', 'watching', '.']

## 1.5 评价翻译结果 ##   
评价机器翻译结果通常使用BLEU（Bilingual Evaluation Understudy）。对于模型预测序列中任意的子序列，BLEU考察这个子序列是否出现在标签序列中。 
  
具体来说，BLEU指标考虑预测序列中与标签序列匹配的不同长度的子序列的精度，而且对匹配较长子序列的精度赋予了更大的权重。BLEU的计算公式中包括一个惩罚项，用以惩罚较短的输出序列，确保对短序列长度的匹配能够进行合理的评估。

在实现BLEU的计算时，我们会使用预测序列和标签序列的词数，计算匹配的子序列数量的归一化精确度。最终的BLEU值是根据这些精度值的几何平均和一个长度惩罚项来计算的。
BLEU（Bilingual Evaluation Understudy）是一个在自然语言处理任务中广泛使用的评价指标，特别是在机器翻译任务中。BLEU 是基于n-gram精度的评测指标，它的基本思想是：如果一个机器翻译的输出和人工翻译（参考答案）越接近，则其翻译质量越好。

具体来看，假设我们的预测序列的长度为$L_p$，参考答案的长度为$L_t$，并且我们只考虑到n个单词的子序列。我们可以定义子序列的精确度 $P_n$：

$$P_n = \frac{\text{Number of $n$-word sequences in both Prediction and Reference}}{\text{Number of $n$-word sequences in Prediction}},$$ 

这是一个相对频率，即预测序列中跟参考答案匹配上的n个单词的子序列数量占预测序列中所有n个单词子序列的比例。

BLEU的计算公式定义如下：

$$ BLEU = BP \cdot \exp \left( \sum_{n=1}^N w_n \log P_n \right),$$ 

其中 $BP$ 是Brevity Penalty的缩写，用于惩罚过短的预测序列。$BP$ 的定义如下：

$$BP = 
\left\{
  \begin{array}{lr}
    1 & : L_p > L_t \\
    \exp \left(1 - \frac{L_t}{L_p}\right) & : L_p \leq L_t
  \end{array}
\right.
$$

在这个公式中，如果预测序列的长度$L_p$小于或等于参考答案的长度$L_t$，则BLEU score的值会被整体缩小。

$w_n$ 是权重，通常我们取 $w_n = \frac{1}{N}$，那么就是相当于各个$n$的精度的几何平均。

一个BLEU分数满分是1（即100%匹配预测和参考序列），但是实际中很少能达到这个分数。含有较多合适的n-grams的翻译会得到较高的分数。    

接下来，我们将开始实现BLEU指标的计算。

In [15]:
import math
import collections

def bleu(pred_tokens, label_tokens, k):
    # 计算预测序列和参考序列的长度
    len_pred, len_label = len(pred_tokens), len(label_tokens)
    # 初始化BLEU分数的值，考虑到长度惩罚项
    score = math.exp(min(0, 1 - len_label / len_pred))
    
    # 循环计算每个n-gram的匹配情况
    for n in range(1, k + 1):
        num_matches, label_subs = 0, collections.defaultdict(int)
        
        # 统计参考序列中所有长度为n的subsequence的出现次数
        for i in range(len_label - n + 1):
            label_subs[''.join(label_tokens[i: i + n])] += 1
        
        # 统计预测序列中与参考序列匹配的子序列数量
        for i in range(len_pred - n + 1):
            if label_subs[''.join(pred_tokens[i: i + n])] > 0:
                num_matches += 1
                label_subs[''.join(pred_tokens[i: i + n])] -= 1
        
        # 将匹配数量应用到BLEU公式中，得出最终的BLEU分数
        score *= math.pow(num_matches / (len_pred - n + 1), math.pow(0.5, n))
    
    # 返回计算得到的BLEU分数
    return score
    
def score(input_seq, label_seq, k):
    # 使用翻译模型对输入序列进行翻译，得到预测的单词序列
    pred_tokens = translate(encoder, decoder, input_seq, max_seq_len)
    # 将参考序列（label_seq）按空格分割为单词，以便进行BLEU分数计算
    label_tokens = label_seq.split(' ')
    # 输出BLEU分数及预测结果
    print('bleu %.3f, predict: %s' % (bleu(pred_tokens, label_tokens, k),
                                      ' '.join(pred_tokens)))


预测正确则分数为1。

In [16]:
score('ils regardent .', 'they are watching .', k=2)

bleu 1.000, predict: they are watching .


In [17]:
score('ils sont canadienne .', 'they are canadian .', k=2)

bleu 0.658, predict: they are actors .
